In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate

from feature_engine.selection import DropCorrelatedFeatures, SmartCorrelatedSelection
from helper import *


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# !pip install feature-engine

In [3]:
# find and remove correlated features

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    
    corr_matrix = dataset.corr()
    
    for i in range(len(corr_matrix.columns)):
    
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

def CorrelationReduction(data,class_name="class",threshold=0.9):
    X=data.drop(labels=[class_name], axis=1)
    Y=data[class_name]
    corr_features = correlation(X,threshold)
    data.drop(labels=corr_features, axis=1, inplace=True)
    return data
    

In [4]:
data = pd.read_csv('onlyAlpha.csv')
data

,Fp1-LE_alpha,F3-LE_alpha,C3-LE_alpha,P3-LE_alpha,O1-LE_alpha,F7-LE_alpha,T3-LE_alpha,T5-LE_alpha,Fz-LE_alpha,Fp2-LE_alpha,...,C4-LE_alpha,P4-LE_alpha,O2-LE_alpha,F8-LE_alpha,T4-LE_alpha,T6-LE_alpha,Cz-LE_alpha,Pz-LE_alpha,A2-A1_alpha,class
0,11.19,7.75,13.34,11.89,6.69,8.49,6.46,7.86,8.31,11.32,...,4.65,6.09,8.18,10.04,2.77,4.52,8.54,9.02,7.85,0
1,14.20,9.54,13.86,16.57,11.09,12.98,6.73,11.64,11.53,14.67,...,9.20,13.80,14.05,16.58,4.62,8.93,11.43,12.49,9.74,0
2,10.63,9.36,6.75,10.46,8.80,6.55,4.70,7.26,11.13,8.55,...,5.91,7.11,7.81,6.04,2.65,4.40,10.43,9.40,8.79,0
3,31.42,20.69,49.08,52.89,17.34,16.68,14.99,25.63,23.69,28.18,...,16.37,27.07,23.36,18.37,5.11,13.56,26.13,36.55,7.69,0
4,11.02,9.65,33.14,34.99,15.42,8.30,10.13,17.58,10.45,11.47,...,10.73,19.07,25.06,8.55,2.64,11.44,16.71,26.95,6.90,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1136,28.04,29.33,26.85,18.75,16.63,21.65,24.31,16.02,32.90,28.56,...,29.19,27.08,20.64,31.56,42.09,27.00,32.34,24.29,43.66,1
1137,31.86,39.36,49.55,37.09,18.11,23.69,43.28,34.24,38.53,34.67,...,34.00,28.34,18.99,29.04,26.23,19.37,39.78,30.92,21.07,1
1138,30.12,21.56,17.70,17.86,13.41,13.71,13.71,14.87,20.09,25.51,...,18.09,15.37,14.16,14.65,10.93,11.75,19.92,16.35,4.84,1
1139,19.72,31.87,35.32,25.38,19.03,20.42,35.75,17.27,22.80,16.07,...,21.05,21.22,18.09,10.88,12.91,14.43,25.57,22.33,9.83,1


In [5]:
data2=CorrelationReduction(data.copy(),class_name="class",threshold=0.85)
data2.shape

(1141, 6)

In [6]:
res=quick_result(data2,class_name="class",test_size=0.20,random_state=42)
res

,Algo Names,Accuracy,Sensitivity,Specificity,F1-Score
0,SVM,0.641921,0.111111,0.985612,0.196078
1,LogisticRegression,0.620087,0.266667,0.848921,0.355556
2,DecissionTree,0.737991,0.777778,0.712230,0.700000
3,RandomForest,0.799127,0.811111,0.791367,0.760417
4,Voting,0.672489,0.211111,0.971223,0.336283


In [7]:
def getCromosome(data1,data2):
    column_names1 = data1.columns.tolist()
    column_names2 = data2.columns.tolist()
    cromo_size=len(column_names1)
    cromo = [0] * cromo_size
    cur=0
    for feature in column_names1:
        if feature in column_names2:
            cromo[cur]=1
        cur=cur+1
    return cromo

In [8]:
firstCromosome=getCromosome(data,data2)
firstCromosome

[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1]

In [9]:
class myResult:
    def __init__(self, result, features, cromosome,data):
        self.result = result
        self.features = features
        self.cromosome = cromosome
        self.data=data

    def __repr__(self):
        return f"MyDataContainer(result={self.result}, features={self.features}, cromosome={self.cromosome},data={self.data})"

def correlationResult(data):
    data2=CorrelationReduction(data.copy(),class_name="class",threshold=0.85)
    features = data2.columns.tolist()
    res=quick_result(data2.copy(),class_name="class",test_size=0.20,random_state=42)
    cromo=getCromosome(data,data2)
    makeResult=myResult(res,features,cromo,data2)
    return makeResult

In [10]:
data = pd.read_csv('onlyAlpha.csv')
res=correlationResult(data)

In [11]:
res.cromosome

[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1]

In [12]:
res.result

,Algo Names,Accuracy,Sensitivity,Specificity,F1-Score
0,SVM,0.641921,0.111111,0.985612,0.196078
1,LogisticRegression,0.620087,0.266667,0.848921,0.355556
2,DecissionTree,0.742358,0.777778,0.719424,0.703518
3,RandomForest,0.794760,0.777778,0.805755,0.748663
4,Voting,0.668122,0.200000,0.971223,0.321429


In [13]:
res.features

['Fp1-LE_alpha',
 'F7-LE_alpha',
 'T3-LE_alpha',
 'C4-LE_alpha',
 'A2-A1_alpha',
 'class']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['class'], axis=1),
    data['class'],
    test_size=0.3,
    random_state=0)

# random forest
rf = RandomForestClassifier(
    n_estimators=10,
    random_state=20,
    n_jobs=4,
)
from feature_engine.selection import DropCorrelatedFeatures, SmartCorrelatedSelection
# correlation selector
sel = SmartCorrelatedSelection(
    variables=None, # if none, selector examines all numerical variables
    method="pearson",
    threshold=0.8,
    missing_values="raise",
    selection_method="model_performance",
    estimator=rf,
    scoring="roc_auc",
    cv=3,
)

# this may take a while, because we are training
# a random forest per correlation group

#sel.fit(X_train, y_train)



In [23]:
group = sel.correlated_feature_sets_[1]

# build random forest with cross validation for
# each feature

for f in group:
    
    model = cross_validate(
        rf,
        X_train[f].to_frame(),
        y_train,
        cv=3,
        return_estimator=False,
        scoring='roc_auc',
    )

    print(f, model["test_score"].mean())

Pz-LE_alpha 0.6646834053001607
T4-LE_alpha 0.5061730477473362
F8-LE_alpha 0.5938962504063175
C4-LE_alpha 0.6879809365379835
Cz-LE_alpha 0.639689481106977
O2-LE_alpha 0.6200379070345514
T6-LE_alpha 0.5863406697958885
P4-LE_alpha 0.6339472288320934


In [24]:
sel.correlated_feature_sets_

[{'C3-LE_alpha',
  'F3-LE_alpha',
  'F4-LE_alpha',
  'F7-LE_alpha',
  'Fp1-LE_alpha',
  'Fp2-LE_alpha',
  'Fz-LE_alpha',
  'P3-LE_alpha'},
 {'C4-LE_alpha',
  'Cz-LE_alpha',
  'F8-LE_alpha',
  'O2-LE_alpha',
  'P4-LE_alpha',
  'Pz-LE_alpha',
  'T4-LE_alpha',
  'T6-LE_alpha'},
 {'O1-LE_alpha', 'T5-LE_alpha'}]

In [26]:
sel.features_to_drop_

['F3-LE_alpha',
 'F4-LE_alpha',
 'F7-LE_alpha',
 'Fp1-LE_alpha',
 'Fp2-LE_alpha',
 'Fz-LE_alpha',
 'P3-LE_alpha',
 'Cz-LE_alpha',
 'F8-LE_alpha',
 'O2-LE_alpha',
 'P4-LE_alpha',
 'Pz-LE_alpha',
 'T4-LE_alpha',
 'T6-LE_alpha',
 'O1-LE_alpha']

In [21]:
sel.fit(X_train, y_train)

SmartCorrelatedSelection(estimator=RandomForestClassifier(n_estimators=10,
                                                          n_jobs=4,
                                                          random_state=20),
                         missing_values='raise',
                         selection_method='model_performance')

In [22]:
X_train.shape

(798, 20)

# Mutual Information

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# to obtain the mutual information values
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

# to select the features
from sklearn.feature_selection import SelectKBest, SelectPercentile
data = pd.read_csv('onlyAlpha.csv')
data